In [85]:
from typing import Tuple


class tensor:
    def __init__(self,size,stride,offset,storage):
        self.size = size
        self.stride = stride
        self.offset = offset
        self.storage = storage
    
    def __repr__(self):
        return "size:"+str(self.size)+"\nstride:"+str(self.stride)+"\noffset:"+str(self.offset)+"\nstorage:"+str(self.storage)
    
    def __getitem__(self, *idxs):
        idx = idxs[0]
        # multiple arguments
        if isinstance(idx, tuple):
            a, b = idx
            if isinstance(a, slice):
                start, stop, step = self._parse_slice(a, 0)
                idx_0 = [self. offset + x * self.stride[0] for x in range(start, stop, step)]
            else:
                idx_0 = [a]
            if isinstance(b, slice):
                start, stop, step = self._parse_slice(b, 1)
                idx_1 = [self.offset + x * self.stride[1] for x in range(start, stop, step)]
            else:
                idx_1 = [b]
        # single argument
        else:
            idx_0 = [idx]
            idx_1 = [self.offset + x * self.stride[1] for x in range(0, self.size[1])] # this is potentially wrong TODO
            
        r = []
        for x0 in idx_0:
            ri = []
            for x1 in idx_1:
                ind = self.offset + x0 + x1
                if ind >= len(self.storage):
                    raise IndexError("Specified Indices can't be selected, either offset is " + 
                        "non zero or indices are to big, figure it out yourself" +
                        f"indices dim0: {idx_0}, indices dim1: {idx_1}, offset: {self.offset}")
                ri.append(self.storage[ind])
            r.append(ri)
        return r

    def _parse_slice(self, slice: slice, dim:int) -> tuple[int, int, int]:
        start = slice.start if slice.start is not None else 0 
        end = slice.stop if slice.stop is not None else self.size[dim] 
        step = slice.step if slice.step is not None else 1 
        return start, end, step
        
    def transpose(self, dim0, dim1) -> None:
        self.stride = [self.stride[dim0], self.stride[dim1]]
        
    def is_contiguous(self) -> bool:
        return self.size[1] == self.stride[0]
        
    def unsqueeze(self,dim=0) -> None:
        if len(self.stride) > 1 or dim > 1 or dim < 0:
            raise Exception("This tensor implementation only supports two 2d tensors, so only unsqueezing of 1d tensors is possible")
        if dim==0:
            self.size = [1, self.size[0]]
            self.stride = [1, 1] # stride is only relevant if we unsqueeze tensors that are > 1d which is not supported
        elif dim==1:
            self.size = [self.size[0], 1]
            self.stride = [1, 1]


stride=[2,1]
size=[5,2]
storage=range(10)
t=tensor(size,stride,0,storage)

print(t[2:4:, ::])
print(t[2])
print(t.is_contiguous())

t.transpose(1, 0)
print(t[::, ::])
print(t[2])
print(t.is_contiguous())

t.transpose(1, 0)
print(t.stride)
print(t.is_contiguous())



[[4, 5], [6, 7]]
[[2, 3]]
True
[[0, 2], [1, 3], [2, 4], [3, 5], [4, 6]]
[[2, 4]]
False
[2, 1]
True


In [86]:
# just some code to help me understand how contiguous works
import torch
a = torch.arange(0, 10).view(5, 2)
print(a, a.stride(), a.is_contiguous(), a.shape)
a = a.T
print(a.is_contiguous())
a = a.T
print(a.is_contiguous())
b = torch.arange(0, 10)
print(b, b.stride())

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]]) (2, 1) True torch.Size([5, 2])
False
True
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) (1,)
